In [1]:
import pandas as pd
import numpy as np

In [5]:
#STATS CELL: This prepares the stats section into a dataframe
#Passing in csv file and getting rid of the unused rows
statsDF = pd.read_csv('stats.csv', delim_whitespace=True)
statsDF.columns = [col.replace(',', '') for col in statsDF.columns]
# if you want to operate on multiple columns, put them in a list like so:
cols = ['PLAYER', 'MIN', 'FGM','FGA', 'FG%', '3PM', '3PA','3P%','FTM','FTA','FT%','OREB','DREB','REB','AST','TOV','STL','BLK','PF','PTS','+/-']
# pass them to df.replace(), specifying each char and it's replacement:
statsDF[cols] = statsDF[cols].replace({'\$': '', ',': ''}, regex=True)
#Remove columns for team name, totals, and the word 'player'
statsDF = statsDF[statsDF.PLAYER != 'PLAYER']
statsDF = statsDF[statsDF.PLAYER != 'Totals:']
statsDF = statsDF[statsDF.PLAYER != 'Cleveland']
statsDF = statsDF[statsDF.PLAYER != 'BOS:']
statsDF = statsDF[statsDF.PLAYER != 'CLE:']
#Reset Index
statsDF = statsDF.reset_index(drop=True)

In [58]:
#GAMES CELL: This prepares the games section into a dataframe
#Passing in csv file and getting rid of the unused rows
gamesDF = pd.read_csv('games.csv', header= None, names=['a','b','c','d','e','f','g','h','i'], index_col=False, error_bad_lines=False, delim_whitespace=True)
gamesDF.columns = [col.replace(',', '') for col in gamesDF.columns]
#if you want to operate on multiple columns, put them in a list like so:
games_cols = ['a','b','c','d','e','f','g','h','i']
# pass them to df.replace(), specifying each char and it's replacement:
gamesDF[games_cols] = gamesDF[games_cols].replace({'\$': '', ',': ''}, regex=True)
# This will add the winner to row 1, colmumn b
if int(gamesDF['a'].iloc[4]) > int(gamesDF['a'].iloc[8]):
    gamesDF['b'].iloc[0] = gamesDF['a'].iloc[1]
else:
    gamesDF['b'].iloc[0] = gamesDF['a'].iloc[5]

In [72]:
#This cell will arrange the gamesDF into the actual formatted games dataframe, newGamesDF
#Setting up the index on the main stats dataframe
dateIndex = [gamesDF.iloc[9,0] + " " + gamesDF.iloc[9,1][1] + ", " + gamesDF.iloc[9,2]]
#Create an empty dataframe, based on the games date as the index 
newGamesDF = pd.DataFrame(index = dateIndex, columns=['Home','Away','W/L','1st Qtr H','2nd Qtr H','3rd Qtr H','4th Qtr H','1st Qtr A','2nd Qtr A','3rd Qtr A','4th Qtr A','Total H','Total A', 'Ref1','Ref2','Ref3'])
#The next lines will pull the data from the cells in the gamesDF and into the newGamesDF dataframe
newGamesDF['Away'] = gamesDF['a'].iloc[1]
newGamesDF['Home'] = gamesDF['a'].iloc[5]
newGamesDF['W/L'] = gamesDF['b'].iloc[0]
newGamesDF['1st Qtr H'] = gamesDF['b'].iloc[12]
newGamesDF['2nd Qtr H'] = gamesDF['c'].iloc[12]
newGamesDF['3rd Qtr H'] = gamesDF['d'].iloc[12]
newGamesDF['4th Qtr H'] = gamesDF['e'].iloc[12]
newGamesDF['1st Qtr A'] = gamesDF['b'].iloc[11]
newGamesDF['2nd Qtr A'] = gamesDF['c'].iloc[11]
newGamesDF['3rd Qtr A'] = gamesDF['d'].iloc[11]
newGamesDF['4th Qtr A'] = gamesDF['e'].iloc[11]
newGamesDF['Total H'] = gamesDF['f'].iloc[12]
newGamesDF['Total A'] = gamesDF['f'].iloc[11]
newGamesDF['Ref1'] = gamesDF['c'].iloc[22][:-2]
newGamesDF['Ref2'] = gamesDF['e'].iloc[22][:-2]
newGamesDF['Ref3'] = gamesDF['g'].iloc[22]
newGamesDF

,Home,Away,W/L,1st Qtr H,2nd Qtr H,3rd Qtr H,4th Qtr H,1st Qtr A,2nd Qtr A,3rd Qtr A,4th Qtr A,Total H,Total A,Ref1,Ref2,Ref3
"OCT 7, 2017",CLEVELAND,BOSTON,CLEVELAND,29,25,18,30,19,19,33,28,102,99,McCutchen,Smith,Forte


In [62]:
#Setting up the index on the main stats dataframe
mainDFIndex = []
for index, row in statsDF.iterrows():
    if index % 2 == 0 and index != 50:
        mainDFIndex.append(str(row['PLAYER']) + " " + str(row['MIN']))
print(mainDFIndex)

['Gordon Hayward', 'Jayson Tatum', 'Al Horford', 'Jaylen Brown', 'Kyrie Irving', 'Marcus Smart', 'Aron Baynes', 'Semi Ojeleye', 'Terry Rozier', 'Shane Larkin', 'Abdel Nader', 'Daniel Theis', 'LeBron James', 'Jae Crowder', 'Kevin Love', 'Dwyane Wade', 'Derrick Rose', 'Tristan Thompson', 'JR Smith', 'Iman Shumpert', 'Jeff Green', 'Kyle Korver', 'Jose Calderon', 'Channing Frye', 'Cedi Osman']


In [63]:
#Setup dicts with the sub categories
newDict = dict()
something = statsDF.T.to_dict().values()
for index, i in enumerate(something):
    if index % 2 != 0:
        newDict[index] = i

In [85]:
#put them in dataframe, index it with the main index, drop nan columns, rename the columns to their proper categories
finalStats = pd.DataFrame.from_dict(newDict,orient='index')
finalStats.index = mainDFIndex
finalStats.drop(['+/-'], axis=1, inplace=True)
finalStats = finalStats.rename(columns={'PLAYER':'MIN','MIN':'FGM', 'FGM':'FGA', 'FGA':'FG%', 'FG%':'3PM', '3PM':'3PA', '3PA':'3P%', '3P%':'FTM', 'FTM':'FTA', 'FTA':'FT%', 'FT%':'REB', 'REB':'DREB', 'DREB':'REB', 'REB':'AST', 'AST':'TOV', 'TOV':'BLK', 'BLK':'PF', 'PF':'PTS', 'PTS':'+/-'})
finalStats

,TOV,PF,+/-,REB,FTA,STL,FGA,BLK,FGM,FT%,FG%,OREB,REB,PTS,AST,3PM,3P%,MIN,FTM,3PA
Gordon Hayward,0,1,3,0,0,0,2,0,1,0.0,50.0,1,1,2,0,0,0.0,5:15,0,1
Jayson Tatum,1,4,6,4,3,0,12,0,5,100,41.7,6,10,14,3,1,50.0,36:32,3,2
Al Horford,0,2,8,0,7,1,7,0,2,71.4,28.6,7,7,9,5,0,0.0,32:07,5,2
Jaylen Brown,3,5,-5,1,2,0,23,2,11,50.0,47.8,5,6,25,0,2,22.2,39:36,1,9
Kyrie Irving,2,4,-1,2,2,0,17,3,8,100,47.1,2,4,22,10,4,44.4,39:21,2,9
Marcus Smart,2,2,-8,0,3,2,16,2,5,66.7,31.3,9,9,12,3,0,0.0,35:03,2,4
Aron Baynes,2,5,-14,2,4,1,2,0,2,50.0,100,3,5,6,1,0,0.0,19:06,2,0
Semi Ojeleye,0,1,-10,0,0,0,2,0,0,0.0,0.0,0,0,0,0,0,0.0,8:39,0,1
Terry Rozier,0,0,4,0,4,0,6,4,2,100,33.3,3,3,9,2,1,33.3,19:32,4,3
Shane Larkin,0,0,2,0,0,0,1,0,0,0.0,0.0,1,1,0,0,0,0.0,4:49,0,1
